# Proyecto 2
## RSNA 2022 Cervical Spine Fracture Detection
**Universidad del Valle de Guatemala**\
**Facultad de Ingeniería**\
**Departamento de Ciencias de la Computación**\
**Data Science**

# Inferencia de Vertebras Cervicales con EfficientNetV2 de PyTorch
---

## Integrantes
- Gustavo Gonzalez
- Pablo Orellana
- Diego Leiva
- Maria Ramirez

---

## Dependencias

In [1]:
from typing import List

try:
    import pylibjpeg
except:
    # Dependencias para extraccion de JPEG y TorchVision
    # Dependencias de https://www.kaggle.com/code/vslaykovsky/rsna-2022-whl
    !mkdir -p /root/.cache/torch/hub/checkpoints/
    !pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
    !pip install /kaggle/input/rsna-2022-whl/{torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl,torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl}

Processing /kaggle/input/rsna-2022-whl/pydicom-2.3.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/pylibjpeg-1.4.0-py3-none-any.whl
Processing /kaggle/input/rsna-2022-whl/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
pydicom is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/rsna-2022-whl/torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl
Processing /kaggle/input/rsna-2022-whl/torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

## Librerias

In [2]:
# Sistema
import gc
import os

# Manejo de Imagenes DICOM
import cv2
import pydicom as dicom

# Visualizaciones
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Manejo de datos
import numpy as np
import pandas as pd

# Scikit Learn para Folds
from sklearn.model_selection import GroupKFold, KFold

# Pytorch y Cuda
import torch
import torchvision as tv
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast
from torchvision.models.feature_extraction import create_feature_extractor


## Constantes

In [3]:
# Configuracion para display de pandas
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


# Efficient Net
WEIGHTS = tv.models.efficientnet.EfficientNet_V2_S_Weights.DEFAULT

# Rutas base
TRAIN_IMAGES_PATH = '../input/rsna-2022-cervical-spine-fracture-detection/train_images'
TEST_IMAGES_PATH = '../input/rsna-2022-cervical-spine-fracture-detection/test_images'
METADATA_PATH = '../input/metadata'
EFFNET_CHECKPOINTS_PATH = '../input/vertebrae-detection-checkpoints'


# Constantes del modelo
EFFNET_MAX_TRAIN_BATCHES = 10000
EFFNET_MAX_EVAL_BATCHES = 1000
ONE_CYCLE_MAX_LR = 0.0004
ONE_CYCLE_PCT_START = 0.3
SAVE_CHECKPOINT_EVERY_STEP = 500
N_MODELS_FOR_INFERENCE = 2

## Configuracion de CUDA

In [4]:
# Configuracion y deteccion de CUDA
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
if DEVICE == 'cuda':
    BATCH_SIZE = 32
else:
    BATCH_SIZE = 2
N_FOLDS = 5

## Carga del Conjunto de datos

In [5]:
def load_dicom(path):
    """
    Carga una imagen DICOM y la convierte a RGB.

    Args:
        path (str): Ruta de la imagen DICOM.
    
    Returns:
        img (np.array): Imagen RGB.
        img (pydicom.dataset.FileDataset): Objeto DICOM.
    """
    img=dicom.dcmread(path)
    img.PhotometricInterpretation = 'YBR_FULL'
    data = img.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data=(data * 255).astype(np.uint8)
    return cv2.cvtColor(data, cv2.COLOR_GRAY2RGB), img

In [6]:
# Se cargan los datos de segmentación
df_seg = pd.read_csv(f'{METADATA_PATH}/segmentation_metadata.csv')

# Se realizan los splits de la data
split = GroupKFold(N_FOLDS)
for k, (train_idx, test_idx) in enumerate(split.split(df_seg, groups=df_seg.StudyInstanceUID)):
    df_seg.loc[test_idx, 'split'] = k

# Se realiza un split aleatorio
split = KFold(N_FOLDS)
for k, (train_idx, test_idx) in enumerate(split.split(df_seg)):
    df_seg.loc[test_idx, 'random_split'] = k

# Se inicializan y obtienen las relaciones de corte
slice_max_seg = df_seg.groupby('StudyInstanceUID')['Slice'].max().to_dict()
df_seg['SliceRatio'] = 0
df_seg['SliceRatio'] = df_seg['Slice'] / df_seg['StudyInstanceUID'].map(slice_max_seg)

# Se inicializan las columnas de predicción
df_seg.sample(10)

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,C3,C4,C5,C6,C7,split,random_split,SliceRatio
9369,1.2.826.0.1.3680043.20647,28,512,512,0.625,-65.00000,-17.29900,-61.656,0,0,0,0,0,0,0,4.0,1.0,0.106464
18888,1.2.826.0.1.3680043.28327,87,512,512,0.625,-76.20000,-35.70000,-41.750,0,1,0,0,0,0,0,4.0,3.0,0.303136
6193,1.2.826.0.1.3680043.1868,19,512,512,0.625,-52.50000,-55.30000,11.827,0,0,0,0,0,0,0,3.0,1.0,0.029096
12131,1.2.826.0.1.3680043.24140,231,512,512,1.000,-59.86320,-52.36328,-651.000,0,0,0,0,0,0,0,3.0,2.0,0.966527
1972,1.2.826.0.1.3680043.12833,108,512,512,1.000,-93.32520,-238.32520,-212.000,0,1,1,0,0,0,0,3.0,0.0,0.413793
2029,1.2.826.0.1.3680043.12833,165,512,512,1.000,-93.32520,-238.32520,-269.000,0,0,0,0,0,1,1,3.0,0.0,0.632184
25406,1.2.826.0.1.3680043.4769,253,512,512,1.000,-116.07700,-32.95288,-670.100,0,0,0,0,0,0,0,1.0,4.0,0.961977
16717,1.2.826.0.1.3680043.26979,43,512,512,0.625,-75.50000,-39.30000,-27.500,0,0,0,0,0,0,0,2.0,2.0,0.144295
3712,1.2.826.0.1.3680043.1542,130,512,512,0.600,-95.34668,-245.84668,-1241.700,0,1,0,0,0,0,0,4.0,0.0,0.303738
28512,1.2.826.0.1.3680043.8024,189,512,512,0.625,-128.50000,-162.90000,-150.000,0,0,0,0,0,1,0,3.0,4.0,0.692308


## Generador de Dataset PyTorch

In [7]:
class VertebraeSegmentDataSet(torch.utils.data.Dataset):
    """
    Clase para cargar el dataset de segmentación de vértebras.
    El dataset debe contener las columnas 'StudyInstanceUID', 'Slice', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'.

    Args:
        df (pd.DataFrame): Dataframe con las columnas 'StudyInstanceUID', 'Slice', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'.
        path (str): Ruta donde se encuentran las imágenes DICOM.
        transforms (torchvision.transforms): Transformaciones a aplicar a las imágenes.

    Returns:
        img (torch.Tensor): Imagen en formato tensor.
        vert_targets (torch.Tensor): Coordenadas de las vértebras.
    """
    def __init__(self, df, path, transforms=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transforms = transforms

    def __getitem__(self, i):
        path = os.path.join(self.path, self.df.iloc[i].StudyInstanceUID, f'{self.df.iloc[i].Slice}.dcm')
        try:
            img = load_dicom(path)[0]
            img = np.transpose(img, (2, 0, 1))  # Pytorch uses (batch, channel, height, width) order. Converting (height, width, channel) -> (channel, height, width)
            if self.transforms is not None:
                img = self.transforms(torch.as_tensor(img))
        except Exception as ex:
            print(ex)
            return None

        if 'C1' in self.df.columns:
            vert_targets = torch.as_tensor(self.df.iloc[i][['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].astype('float32').values)
            return img, vert_targets
        return img

    def __len__(self):
        return len(self.df)

# Cargar el dataset de segmentación de vértebras con PyTorch
ds_seg = VertebraeSegmentDataSet(df_seg, TRAIN_IMAGES_PATH, WEIGHTS.transforms())
X, y = ds_seg[300]
X.shape, y.shape

(torch.Size([3, 384, 384]), torch.Size([7]))

## Modelo de deteccion de Vertebras con EfficientNetV2

In [8]:
class SegEffnetModel(torch.nn.Module):
    """
    Modelo de clasificación de vértebras con EfficientNetV2
    Este modelo se encarga de clasificar las vértebras de una radiografía
    en base a una red EfficientNetV2 preentrenada.

    Args:
        None
    """
    def __init__(self):
        super().__init__()
        # Inicialización de modelo EfficientNetV2 preentrenado
        # con pesos de imagenet y arquitectura small
        effnet = tv.models.efficientnet_v2_s(weights=WEIGHTS)
        self.model = create_feature_extractor(effnet, ['flatten'])
        self.nn_vertebrae = torch.nn.Sequential(
            torch.nn.Linear(1280, 7),
        )

    def forward(self, x):
        # Retorna logits
        x = self.model(x)['flatten']
        return self.nn_vertebrae(x)

    def predict(self, x):
        # Retorna probabilidades de clase
        pred = self.forward(x)
        return torch.sigmoid(pred)

## Entrenamiento y Evaluacion del modelo
- Se cargan los modelos de manera manual
- Se estima la precision del modelo
- Se usan 5 Modelos con GroupKFold
- Se utiliza el calendarizaro OneCycleLR para entrenamiento de una epoca


### Funciones Auxliares

In [9]:
def gc_collect():
    """
    Limpia la memoria de objetos no utilizados.
    """
    gc.collect()
    torch.cuda.empty_cache()

In [10]:
def filter_nones(b):
    """
    Filtra los elementos nulos de un batch de datos

    Args:
        b (list): Lista de datos
    """
    return torch.utils.data.default_collate([v for v in b if v is not None])

### Funciones Almacenado y Carga de Modelos

In [11]:
def save_model(name, model, optim, scheduler):
    """
    Guarda un modelo en disco.

    Args:
        name: nombre del archivo
        model: modelo a guardar
        optim: optimizador del modelo
        scheduler: scheduler del modelo

    Returns:
        None
    """
    torch.save({
        'model': model.state_dict(),
        'optim': optim.state_dict(),
        'scheduler': scheduler
    }, f'{name}.tph')
    

def load_model(model, name, path='.'):
    """
    Carga un modelo desde disco.

    Args:
        model: modelo a cargar
        name: nombre del archivo
        path: ruta del archivo. Por defecto es el directorio actual.

    Returns:
        model: modelo cargado
        optim: optimizador del modelo
        scheduler: scheduler del modelo
    """
    data = torch.load(os.path.join(path, f'{name}.tph'), map_location=DEVICE)
    model.load_state_dict(data['model'])
    optim = torch.optim.Adam(model.parameters())
    optim.load_state_dict(data['optim'])
    return model, optim, data['scheduler']

# Prueba de la función
model = torch.nn.Linear(2, 1)
optim = torch.optim.Adam(model.parameters())
save_model('testmodel', model, optim, None)

# Carga del Modelo
model1, optim1, scheduler1 = load_model(torch.nn.Linear(2, 1), 'testmodel')
assert torch.all(next(iter(model1.parameters())) == next(iter(model.parameters()))).item(), "Loading/saving is inconsistent!"

### Funcion de Evaluacion del Modelo

In [12]:
def evaluate_segeffnet(model: SegEffnetModel, ds, max_batches=1e9, shuffle=False):
    """
    Evalúa un modelo de segmentación en un dataset.

    Args:
        model (SegEffnetModel): Modelo a evaluar.
        ds (torch.utils.data.Dataset): Dataset a evaluar.
        max_batches(int): Número máximo de batches a evaluar.
        shuffle(bool): Si se debe mezclar el dataset.

    Returns:
        float: Accuracy promedio.
        np.ndarray
    """
    # Se establece el dispositivo de evaluación con semilla fija para reproducibilidad
    torch.manual_seed(42)
    model = model.to(DEVICE)
    # Se crea un DataLoader para el dataset
    dl_test = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE, shuffle=shuffle, num_workers=os.cpu_count(), collate_fn=filter_nones)
    # Se evalúa el modelo en el dataset
    with torch.no_grad():
        model.eval()
        pred = []
        y = []
        # Se itera sobre los batches del dataset
        progress = tqdm(dl_test, desc='Eval', miniters=100)
        for i, (X, y_vert) in enumerate(progress):
            with autocast():
                y_vert_pred = model.predict(X.to(DEVICE))
            # Se almacenan las predicciones y las etiquetas
            pred.append(y_vert_pred.cpu().numpy())
            y.append(y_vert.numpy())
            # Se calcula la precisión del modelo
            acc = np.mean(np.mean((pred[-1] > 0.5) == y[-1], axis=0))
            # Se actualiza la barra de progreso
            progress.set_description(f'Epoch {i} - Eval acc: {acc:.02f}')
            if i >= max_batches:
                break
        # Se calcula la precisión promedio
        pred = np.concatenate(pred)
        y = np.concatenate(y)
        acc = np.mean(np.mean((pred > 0.5) == y, axis=0))
        return acc, pred

### Funcion de Entrenamiento del Modelo

In [13]:
def train_segeffnet(ds_train, ds_eval, logger, name):
    """
    Entrena un modelo de segmentación de vértebras a partir de un dataset de entrenamiento 
    y otro de evaluación.

    Args:
        ds_train (VertebraeSegmentDataSet): Dataset de entren
        ds_eval (VertebraeSegmentDataSet): Dataset de evaluación
        logger (Logger): Logger para almacenar los resultados
        name (str): Nombre del modelo

    Returns:
        SegEffnetModel: Modelo entrenado
    """
    # Configuración de la semilla y el dataloader
    torch.manual_seed(42)
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=os.cpu_count(), collate_fn=filter_nones)

    # Inicialización del modelo, optimizador y scheduler
    model = SegEffnetModel().to(DEVICE)
    optim = torch.optim.Adam(model.parameters())
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, max_lr=ONE_CYCLE_MAX_LR, epochs=1, steps_per_epoch=min(EFFNET_MAX_TRAIN_BATCHES, len(dl_train)), pct_start=ONE_CYCLE_PCT_START)
    model.train()
    scaler = GradScaler()

    # Entrenamiento del modelo
    progress = tqdm(dl_train, desc='Train', miniters=10)
    for batch_idx, (X,  y_vert) in enumerate(progress):

        # Evaluación del modelo cada SAVE_CHECKPOINT_EVERY_STEP pasos
        if batch_idx % SAVE_CHECKPOINT_EVERY_STEP == 0 and EFFNET_MAX_EVAL_BATCHES > 0:
            eval_loss = evaluate_segeffnet(model, ds_eval, max_batches=EFFNET_MAX_EVAL_BATCHES, shuffle=True)[0]
            model.train()
            if logger is not None:
                logger.log({'eval_acc': eval_loss})
            if batch_idx > 0:  # don't save untrained model
                save_model(name, model, optim, scheduler)

        if batch_idx >= EFFNET_MAX_TRAIN_BATCHES:
            break
        
        # Optimización del modelo
        optim.zero_grad()
        with autocast():
            # Forward pass
            y_vert_pred = model.forward(X.to(DEVICE))
            loss = torch.nn.functional.binary_cross_entropy_with_logits(y_vert_pred, y_vert.to(DEVICE))

            # Verificación de la pérdida
            if np.isinf(loss.item()) or np.isnan(loss.item()):
                print(f'Bad loss, skipping the batch {batch_idx}')
                del y_vert_pred, loss
                gc_collect()
                continue
        
        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optim)
        scaler.update()
        scheduler.step()

        # Actualización de la barra de progreso
        progress.set_description(f'Train loss: {loss.item():.02f}')
        if logger is not None:
            logger.log({'loss': loss.item(), 'lr': scheduler.get_last_lr()[0]})


    # Evaluación final del modelo
    eval_loss = evaluate_segeffnet(model, ds_eval, max_batches=EFFNET_MAX_EVAL_BATCHES, shuffle=True)[0]
    if logger is not None:
        logger.log({'eval_acc': eval_loss})

    # Guardado del modelo
    save_model(name, model, optim, scheduler)
    return model

In [14]:
# Configuración de los pesos de las vértebras
seg_models = []
for fold in range(N_FOLDS):
    fname = os.path.join(f'{EFFNET_CHECKPOINTS_PATH}/segeffnetv2-f{fold}.tph')
    if os.path.exists(fname):
        print(f'Found cached model {fname}')
        seg_models.append(load_model(SegEffnetModel(), f'segeffnetv2-f{fold}', EFFNET_CHECKPOINTS_PATH)[0].to(DEVICE))
    else:
        gc_collect()
        ds_train = VertebraeSegmentDataSet(df_seg.query('split != @fold'), TRAIN_IMAGES_PATH, WEIGHTS.transforms())
        ds_eval = VertebraeSegmentDataSet(df_seg.query('split == @fold'), TRAIN_IMAGES_PATH, WEIGHTS.transforms())
        train_segeffnet(ds_train, ds_eval, None, f'segeffnetv2-f{fold}')

Found cached model ../input/vertebrae-detection-checkpoints/segeffnetv2-f0.tph


Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth


  0%|          | 0.00/82.7M [00:00<?, ?B/s]

Found cached model ../input/vertebrae-detection-checkpoints/segeffnetv2-f1.tph
Found cached model ../input/vertebrae-detection-checkpoints/segeffnetv2-f2.tph
Found cached model ../input/vertebrae-detection-checkpoints/segeffnetv2-f3.tph
Found cached model ../input/vertebrae-detection-checkpoints/segeffnetv2-f4.tph


In [15]:
# Unión de los datos
with tqdm(seg_models, desc='Fold') as progress:
    # Iteramos sobre los modelos
    for fold, model in enumerate(progress):
        # Evaluamos el modelo en cada fold
        ds = VertebraeSegmentDataSet(df_seg.query('split == @fold'), TRAIN_IMAGES_PATH, WEIGHTS.transforms())
        acc, pred = evaluate_segeffnet(model, ds, max_batches=1e9, shuffle=False)
        df_seg.loc[df_seg[df_seg.split == fold].index, ['C1_pred', 'C2_pred', 'C3_pred', 'C4_pred', 'C5_pred', 'C6_pred', 'C7_pred']] = pred
        progress.set_description(f'Accuracy: {acc}')

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

Eval:   0%|          | 0/185 [00:00<?, ?it/s]

Eval:   0%|          | 0/190 [00:00<?, ?it/s]

Eval:   0%|          | 0/185 [00:00<?, ?it/s]

Eval:   0%|          | 0/185 [00:00<?, ?it/s]

Eval:   0%|          | 0/190 [00:00<?, ?it/s]

In [16]:
acc = (df_seg[[f'C{i}_pred' for i in range(1, 8)]] > 0.5).values == (df_seg[[f'C{i}' for i in range(1, 8)]] > 0.5).values
print('Effnetv2 accuracy por vertebra', np.mean(acc, axis=0))
print('Effnetv2 accuracy promedio', np.mean(np.mean(acc, axis=0)))

Effnetv2 accuracy por vertebra [0.96094798 0.94056718 0.95880263 0.96195361 0.95169617 0.94656744
 0.9484111 ]
Effnetv2 accuracy promedio 0.9527065854499482


## Inferencia de vertebras para todo el dataset
Se hace un promedio de los outputs de los 5 modelos entrenados.

#### Funcion de predicciones

In [17]:
def predict_vertebrae(df, seg_models: List[SegEffnetModel]):
    """
    Predice las probabilidades de cada una de las clases de vértebras para un conjunto de imágenes.

    Args:
        df: DataFrame con las columnas 'ImageID' y 'StudyInstanceUID'.
        seg_models: Lista de modelos de segmentación de vértebras.

    Returns:
        Un arreglo de NumPy con las probabilidades de cada clase de vértebra para cada imagen.
    """
    # Cargar las imágenes
    df = df.copy()
    ds = VertebraeSegmentDataSet(df, TRAIN_IMAGES_PATH, WEIGHTS.transforms())
    # Crear el DataLoader
    dl_test = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=os.cpu_count(), collate_fn=filter_nones)
    predictions = []
    # Realizar las predicciones para cada imagen
    with torch.no_grad():
        with tqdm(dl_test, desc='Eval', miniters=10) as progress:
            for i, X in enumerate(progress):
                with autocast():
                    pred = torch.zeros(len(X), 7).to(DEVICE)
                    for model in seg_models:
                        pred += model.predict(X.to(DEVICE)) / len(seg_models)
                    predictions.append(pred)
    predictions = torch.concat(predictions).cpu().numpy()
    return predictions

In [18]:
# Se carga la metadata previa
df_train = pd.read_csv(os.path.join(METADATA_PATH, 'dicom_metadata.csv'))

In [19]:
# Se generan las predicciones
pred = predict_vertebrae(df_train, seg_models[:N_MODELS_FOR_INFERENCE])

Eval:   0%|          | 0/22238 [00:00<?, ?it/s]

In [20]:
# Se genera un dataframe con las predicciones de vertebras
df_train[[f'C{i}' for i in range(1, 8)]] = pred
df_train.to_csv('vertebrae_metadataEffNet.csv', index=False)

In [21]:
df_train.head()

,StudyInstanceUID,Slice,ImageHeight,ImageWidth,SliceThickness,ImagePositionPatient_x,ImagePositionPatient_y,ImagePositionPatient_z,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.10001,1,512,512,0.625,-52.308,-27.712,7.282,0.005208,0.015055,0.003290,0.000895,0.001614,0.004632,0.002570
1,1.2.826.0.1.3680043.10001,2,512,512,0.625,-52.308,-27.712,6.657,0.005675,0.015945,0.002117,0.000562,0.001265,0.005648,0.004136
2,1.2.826.0.1.3680043.10001,3,512,512,0.625,-52.308,-27.712,6.032,0.006624,0.030292,0.003079,0.000677,0.000882,0.002238,0.002877
3,1.2.826.0.1.3680043.10001,4,512,512,0.625,-52.308,-27.712,5.407,0.006572,0.022294,0.003103,0.000975,0.001164,0.002434,0.002715
4,1.2.826.0.1.3680043.10001,5,512,512,0.625,-52.308,-27.712,4.782,0.004945,0.022388,0.003236,0.000826,0.000662,0.002304,0.006849
